In [1]:
import pandas as pd
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata

In [2]:
def date_time(table_cells):
    return [data_time.strip() for data_time in list(table_cells.strings)][0: 2]

def booster_version(table_cells):
    out = " ".join([booster_version for i, booster_version in enumerate( table_cells.strings) if i % 2 == 0][0: -1])
    return out

def landing_status(table_cells):
    out = [i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass = unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass = mass[0: mass.find("kg") + 2]
    else:
        new_mass = 0
    return new_mass


def extract_column_from_header(row):
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = " ".join(row.contents)
    
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name 

In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
# TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [5]:
page = requests.get(static_url)

In [6]:
soup = BeautifulSoup(page.text, "html.parser")

In [7]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [8]:
# TASK 2: Extract all column/variable names from the HTML table header

In [9]:
html_tables=soup.find_all("table")

In [10]:
first_launch_table = html_tables[2]
first_launch_table

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

In [11]:
column_names = []

for i in first_launch_table.find_all("th"):
    if extract_column_from_header(i) != None:
        if len(extract_column_from_header(i)) > 0:
            column_names.append(extract_column_from_header(i))

In [12]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome']

In [13]:
# TASK 3: Create a data frame by parsing the launch HTML tables

In [14]:
launch_dict= dict.fromkeys(column_names)

del launch_dict["Date and time ( )"]

launch_dict["Flight No."] = []
launch_dict["Launch site"] = []
launch_dict["Payload"] = []
launch_dict["Payload mass"] = []
launch_dict["Orbit"] = []
launch_dict["Customer"] = []
launch_dict["Launch outcome"] = []
launch_dict["Version Booster"]=[]
launch_dict["Booster landing"]=[]
launch_dict["Date"]=[]
launch_dict["Time"]=[]

In [15]:
extracted_row = 0

for table_number, table in enumerate(soup.find_all("table", "wikitable plainrowheaders collapsible")):
    for rows in table.find_all("tr"):
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        row=rows.find_all("td")
        if flag:
            extracted_row += 1
            launch_dict["Flight No."].append(flight_number)
            print(flight_number)
            datatimelist = date_time(row[0])
            
            # Date value
            date = datatimelist[0].strip(",")
            launch_dict["Date"].append(date)
            print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict["Time"].append(time)
            print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict["Version Booster"].append(bv)
            print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict["Launch site"].append(launch_site)
            print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict["Payload"].append(payload)
            print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict["Payload mass"].append(payload_mass)
            print(payload_mass)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict["Orbit"].append(orbit)
            print(orbit)
            
            # Customer
            if row[6].a != None:
                customer = row[6].a.string
            else: 
                customer = "None"
            launch_dict["Customer"].append(customer)
            print(customer)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict["Launch outcome"].append(launch_outcome)
            print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict["Booster landing"].append(booster_landing)
            print(booster_landing)
            
            print("************************")

1
4 June 2010
18:45
F9 v1.0 B0003.1
CCAFS
Dragon Spacecraft Qualification Unit
0
LEO
SpaceX
Success

Failure
************************
2
8 December 2010
15:43
F9 v1.0 B0004.1
CCAFS
Dragon
0
LEO
NASA
Success
Failure
************************
3
22 May 2012
07:44
F9 v1.0 B0005.1
CCAFS
Dragon
525 kg
LEO
NASA
Success
Not attempted

************************
4
8 October 2012
00:35
F9 v1.0 B0006.1
CCAFS
SpaceX CRS-1
4,700 kg
LEO
NASA
Success

No attempt
************************
5
1 March 2013
15:10
F9 v1.0 B0007.1
CCAFS
SpaceX CRS-2
4,877 kg
LEO
NASA
Success

Not attempted

************************
6
29 September 2013
16:00
F9 v1.1 B1003
VAFB
CASSIOPE
500 kg
Polar orbit
MDA
Success
Uncontrolled
************************
7
3 December 2013
22:41
F9 v1.1
CCAFS
SES-8
3,170 kg
GTO
SES
Success
Not attempted
************************
8
6 January 2014
22:06
F9 v1.1
CCAFS
Thaicom 6
3,325 kg
GTO
Thaicom
Success
Not attempted
************************
9
18 April 2014
19:25
F9 v1.1
Cape Canaveral
SpaceX CRS-3


In [16]:
df = pd.DataFrame(launch_dict)

In [17]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0 B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0 B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0 B0005.1,Not attempted\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0 B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0 B0007.1,Not attempted\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5 B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,F9 B5 B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,F9 B5 B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,F9 B5 B1067.1,Success,3 June 2021,17:29


In [18]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0 B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0 B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0 B0005.1,Not attempted\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0 B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0 B0007.1,Not attempted\n,1 March 2013,15:10


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121 entries, 0 to 120
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Flight No.       121 non-null    object
 1   Launch site      121 non-null    object
 2   Payload          121 non-null    object
 3   Payload mass     121 non-null    object
 4   Orbit            121 non-null    object
 5   Customer         120 non-null    object
 6   Launch outcome   121 non-null    object
 7   Version Booster  121 non-null    object
 8   Booster landing  121 non-null    object
 9   Date             121 non-null    object
 10  Time             121 non-null    object
dtypes: object(11)
memory usage: 10.5+ KB


In [20]:
df["Launch outcome"]

0      Success\n
1        Success
2        Success
3      Success\n
4      Success\n
         ...    
116    Success\n
117    Success\n
118    Success\n
119    Success\n
120    Success\n
Name: Launch outcome, Length: 121, dtype: object

In [21]:
df.to_csv("spacex_web_scraped.csv", index=False)